In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#ativar no google colab
#!unzip fer2013.zip 

: 

sempre que corro esta célula a outra mais abaixo com os valores de labels troca os resultados.... preocupante?!!!!!!!!!!!!!!!

In [77]:
#PRE-PROCESSAMENTO
transform = transforms.Compose([
    transforms.Grayscale(),         #garante que as imagens são a preto e branco
    transforms.Resize((48, 48)),    #garante que têm dimensão 48x48
    transforms.ToTensor(),          #converte para tensores pytorch (1,48,48)
    transforms.Normalize((0.5,), (0.5,))  # normaliza pixels entre -1 e 1 (mean = 0.5, std = 0.5)
])

# "SPLIT" DATASET (definimos o treino e o teste):
#ImageFolder → usa os nomes das pastas como labels
train_dataset = datasets.ImageFolder( root="fer2013/train", transform=transform)
test_dataset = datasets.ImageFolder( root="fer2013/test", transform=transform)

#LOAD data
train_loader = DataLoader( train_dataset, batch_size=64, shuffle=True)  #---------------------------apagavel
test = DataLoader( test_dataset, batch_size=64, shuffle=False)

labels correspondem ás emoçoes ou seja

imagens -> x

labels -> y

In [76]:
#VERIFICAR SE ESTÁ TUDO OK--------------------------------------apagavel
images, labels = next(iter(train_loader))
print(images.shape)  # deve dar: [64, 1, 48, 48]
print(labels.shape)  # deve dar: [64]
print(images.min(), images.max())  # já está normalizado para -1 a 1


torch.Size([64, 1, 48, 48])
torch.Size([64])
tensor(-1.) tensor(1.)


0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [78]:
print(f"Labels values (y): \n {labels}")
print(f"image vector (x): \n {images[0,0,:5,:5]}") # mostra os primeiros 5x5 pixels da primeira imagem

Labels values (y): 
 tensor([5, 0, 6, 4, 0, 6, 0, 3, 3, 5, 0, 0, 3, 0, 2, 0, 5, 3, 3, 2, 2, 4, 3, 3,
        0, 2, 4, 3, 3, 3, 4, 5, 4, 6, 0, 3, 3, 3, 5, 5, 3, 5, 6, 2, 3, 4, 3, 4,
        3, 3, 6, 3, 6, 6, 5, 5, 2, 3, 3, 3, 5, 0, 1, 6])
image vector (x): 
 tensor([[0.8353, 0.8353, 0.7255, 0.6784, 0.6941],
        [0.8353, 0.8118, 0.7176, 0.6863, 0.6941],
        [0.8431, 0.7961, 0.7176, 0.7020, 0.6941],
        [0.8510, 0.7804, 0.7333, 0.7176, 0.6784],
        [0.8353, 0.7569, 0.7333, 0.7098, 0.6549]])


In [ ]:
#MONTE CARLO CV---------------------------------------------
from sklearn.model_selection import StratifiedShuffleSplit

def split_train_val_stratified_wrong(train_dataset, k_folds, val_ratio, batch_size, seed):
    targets = train_dataset.targets  #lista de labels (0, 1, 2, ....)

    skf = StratifiedShuffleSplit(
        n_splits=k_folds,
        test_size=val_ratio,
        random_state=seed,
        shuffle=True)

    train_idx, val_idx = next(skf.split(np.zeros(len(targets)), targets))

    train_subset = Subset(train_dataset, train_idx)
    val_subset   = Subset(train_dataset, val_idx)

    train = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    return train, val


In [80]:
#VERDADEIRO K-FOLD----------------------------------------------
from sklearn.model_selection import StratifiedKFold

def split_train_val_stratified(train_dataset, k_folds, batch_size, seed):
    targets = np.array(train_dataset.targets)  #labels

    skf = StratifiedKFold(
        n_splits=k_folds,
        shuffle=True,
        random_state=seed)

    loaders = []

    for train_idx, val_idx in skf.split(np.zeros(len(targets)), targets):
        train_subset = Subset(train_dataset, train_idx)
        val_subset   = Subset(train_dataset, val_idx)

        train = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
        val= DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        loaders.append((train, val))

    return loaders


In [86]:
#do model CNN
num_emotions = 7
lr = 0.001
num_epochs = 100
dropout = 0.1 
batch_size = 64
k_folds = 3
patience =10

out_channels = nº filtros
kenerl = tamanho do filtro
filtro = conjunto de pesos aprendidos

In [87]:
class CNN(nn.Module):
    #def __init__(self, num_classes = num_emotions):
    def __init__(self, num_classes = num_emotions, dropout_prob = dropout): 
        super(CNN, self).__init__()
        #1ª camada convolucional
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)   # (1,48,48) → (16,48,48)
        #2ª camada convolucional
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)  # (16,24,24) → (32,24,24)                         
        #3ª camada convolucional
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # (32,12,12) → (64,12,12)
        self.pool = nn.MaxPool2d(2,2)  # halves spatial size
        self.dropout = nn.Dropout(dropout_prob)
        
        #fully connected
        self.fc1 = nn.Linear(64 * 6 * 6, 256)  # calcula o tamanho correto: 64 filtros, 6x6 depois do pooling
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):      # input x shape: (batch_size, 1, 48, 48)
        x = self.pool(F.relu(self.conv1(x)))  # conv1 + relu + pool: (batch,16,48,48) → (batch,16,24,24)
        x = self.pool(F.relu(self.conv2(x)))  # conv2 + relu + pool: (batch,32,24,24) → (batch,32,12,12)
        x = self.pool(F.relu(self.conv3(x)))  # conv3 + relu + pool: (batch,64,12,12) → (batch,64,6,6)
        x = torch.flatten(x, 1)     # flatten except batch dim
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)       #output logits
        return x

Bons candidatos para tuning:

-lr
-dropout
-nº de filtros por layer
-nº de camadas


-kernel size (às vezes)????? + padding tem valores padrao é ok

-Repeated Hold-Out

-Monte Carlo Cross-Validation

como se pode chamar ao tipo de cross validation que estou a fazer, pois uso um valor de seed variavel

In [88]:
#CROSS VALIDATION
def cross_validation_cnn(
    train_dataset,
    num_emotions,
    device,
    k_folds = k_folds,
    patience = patience,
    batch_size= batch_size,
    num_epochs = num_epochs,
    lr = lr,
    dropout_prob= dropout,
):
    fold_results = []
    
    #opção certa de k-fold verdadeiro
    fold_loaders = split_train_val_stratified(train_dataset, k_folds, batch_size, seed = 42)
    
    for fold, (train, val) in enumerate(fold_loaders):
        print(f"\n===== Fold {fold+1}=====")
    #for run in range(k_folds):
        #print(f"\n===== Fold {run+1}/{k_folds} =====")
        
        #split train  val
        #seed = 42 + run   #“Different random seeds were used at each fold to ensure variability in the train–validation splits.”
        #val_ratio = 0.2

        
        
        #opção com stratify MONTE CARLO
        #train, val = split_train_val_stratified_wrong(train_dataset, k_folds, val_ratio, batch_size=batch_size, seed=seed)

    
        #MODEL (novo em cada k-fold)
        model = CNN (num_classes = num_emotions, dropout_prob = dropout_prob).to(device)
        criterion = nn.CrossEntropyLoss()    #for multi-class classification
        optimizer = optim.Adam(model.parameters(), lr = lr)

        #early stopping (por fold)
        best_val_loss = float("inf") #infinito positivo, para ser maior que qualquer loss possível
        best_val_acc = 0
        counter = 0


        for epoch in range(num_epochs):
            #----TRAIN----
            model.train()
            #train_loss=0
            correct, total = 0, 0
                        
            for images, labels in train:  #images = batch_x, labels = batch_y
                images, labels = images.to(device), labels.to(device)

                logits = model(images)    #logits =outputs
                loss = criterion(logits, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                #train_loss += loss.item() * images.size(0)

                _, preds = torch.max(logits, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            #train_loss /= total
            #train_acc = correct / total
        
            #----VALIDATION----
            model.eval()
            val_loss, val_correct, val_total = 0, 0, 0
    
            with torch.no_grad():
                for images, labels in val:
                    images, labels = images.to(device), labels.to(device)

                    logits = model(images)
                    loss = criterion(logits, labels)

                    val_loss += loss.item() * images.size(0)
                    _, preds = torch.max(logits, 1)
                    val_correct += (preds == labels).sum().item()
                    val_total += labels.size(0)

                val_loss /= val_total
                val_acc = val_correct / val_total

                print(
                    f"Epoch [{epoch+1}/{num_epochs}] "
                    #f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
                    f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
                #----EARLY STOPPING----  
                if val_loss < best_val_loss:
                   best_val_loss = val_loss
                   best_val_acc = val_acc
                   counter = 0
            
                else:
                    counter += 1
                    if counter >= patience:  
                       print("Early stopping triggered")
                       break

        fold_results.append(best_val_acc)
    
    mean_acc = sum(fold_results) / len(fold_results)
    print(f"\nMean validation accuracy over {k_folds} folds: {mean_acc:.4f}")

    return mean_acc     

na função de cima não estou a guardar o best model por fold apenas o best-val_acc.... os pesos não ficam guardados. ok para tuning mas para o futuro temos de ver se não tenho de acrescentar.

In [89]:
#hyperparameters tunning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_overall_acc = 0
best_hyperparams = {}

for lr in [0.001, 1e-4]:
    for dropout in [0.0, 0.2]:
        print(f"\nTesting lr={lr}, dropout={dropout}")
        mean_acc = cross_validation_cnn(
            train_dataset,
            num_emotions,
            device,
            k_folds= k_folds,       #no anterior temos tres par acelerar
            lr=lr,
            dropout_prob =dropout)
        
        if mean_acc > best_overall_acc:
            best_overall_acc = mean_acc
            best_hyperparams = {"lr": lr, "dropout": dropout}

print("\nBest mean validation accuracy:", best_overall_acc)
print("Best hyperparameters:", best_hyperparams)



Testing lr=0.001, dropout=0.0

===== Fold 1=====


KeyboardInterrupt: 

In [ ]:
#Final training with best hyperparameter
#----------------------------------------------
# TREINO FINAL COM MELHORES HYPERPARAMETERS--------por rever-----------------
#----------------------------------------------

final_model = CNN(num_classes=num_emotions, dropout_prob=best_hyperparams['dropout']).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_hyperparams['lr'])


for epoch in range(num_epochs):
    final_model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

# salvar modelo final
torch.save(final_model.state_dict(), "best_cnn_model.pth")
print("Final model saved as 'best_cnn_model.pth'")

: 

após treinar no google colab

In [ ]:
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()


: 

In [ ]:
#TESTE THE MODEL
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images_x, labels_y in test:
        images_x = images_x.to(device)
        labels_y = labels_y.to(device)
        logits = model(images_x)
        _, preds = torch.max(logits, 1)
        
        #all_preds.extend(preds.cpu().numpy())    -- converte para numpy pior se quiser continuar a usar pytorch/GPU
        all_preds.append(preds)
        all_labels.append(labels)  

# concatenar todos os batches em um tensor só
all_preds = torch.cat(all_preds).cpu()
all_labels = torch.cat(all_labels).cpu()

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")       


: 

: 

In [ ]:
#CLASSIFICATION REPORT (stor way)
print("Classification Report:")
print(classification_report(all_labels, all_preds))

: 

: 

In [ ]:
#CLASSIFICATION REPORT (CHAT WAY)
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=[
    'Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'
]))

: 

: 

In [ ]:
#FUZZY PART------------------------------------------------------

probs = torch.softmax(logits, dim = 1)  # [batc_size, 7]
#CRIAR ALGO GENERICO QUE DÉ PARA TODAS AS EMOÇOEES SEM TER QUE ESCREVR UMA POR UMA
'Obs: mais avançado seria usar trapezoidal ou triangular membership functions, mas para o projeto de aula isto já é suficiente para interpretabilidade'

def fuzzy_happy(prob):
    if prob < 0.3:
        return "slightly happy"
    elif prob < 0.7:
        return "moderately happy"
    else:
        return "highly happy"

# probs → saída softmax da CNN
softmax_probs = torch.softmax(logits, dim=1).cpu().numpy()[0]  # exemplo de 1 imagem

labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
fuzzy_labels = []

for i, label in enumerate(labels):
    p = softmax_probs[i]
    if label == 'Happy':
        fuzzy_labels.append(fuzzy_happy(p))
    # repetir para outras emoções com suas funções fuzzy

print(fuzzy_labels)



: 

: 